In [1]:
import cv2
import matplotlib.pyplot as plt

__Cascade Classifiers 로드__

In [2]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontface.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
glass_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')

__advanced_eye_detect 함수 정의__

In [44]:
def advanced_eye_detect(img,cascade,info):
    
    ret_val=None
    
    img_gray =  cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
    eyes =cascade.detectMultiScale(img_gray,1.1,4)
    
    if (len(eyes)==0) or (eyes.shape[0]<2):
        
        if info == 'normal':
            return advanced_eye_detect(img,glass_cascade,'glasses')
        elif info == 'glasses':
            return ret_val
        
    ret_val = eyes
    return ret_val

__get_aligned_ROI 함수 정의__

In [43]:
def get_aligned_ROI(img , eyes , early_stop=False):
    
    eye_1 , eye_2 = eyes

    if eye_1[0] < eye_2[0]:
        left_eye = eye_1
        right_eye = eye_2
    else:
        left_eye = eye_2
        right_eye = eye_1
        
    left_eye_center = (int(left_eye[0] + (left_eye[2] / 2)), int(left_eye[1] + (left_eye[3] / 2)))
    right_eye_center = (int(right_eye[0] + (right_eye[2]/2)), int(right_eye[1] + (right_eye[3]/2)))
    
    left_eye_x , left_eye_y = left_eye_center 
    right_eye_x , right_eye_y = right_eye_center

    
    delta_x = right_eye_x - left_eye_x
    delta_y = right_eye_y - left_eye_y
    
    if early_stop:
        return left_eye_x , left_eye_y , np.sqrt(delta_x**2 + delta_y**2)
    
    angle=np.arctan(delta_y/delta_x)
    angle = (angle * 180) / np.pi
    
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, (angle), 1.0)
    rotated_img = cv2.warpAffine(img, M, (w, h))
    
    rotated_eyes =eye_cascade.detectMultiScale(rotated_img,1.1,4)
    rotated_eyes
    
    
    return rotated_eyes , rotated_img

__메인 코드__

In [ ]:
import os

img_path = "전체 사진이 모여있는 폴더 경로명"
files = os.listdir(img_path)
images = [file for file in files if file.endswith(".png")]

for image in images:
    
    if image.shape[0]<400 or image.shape[1]<400 :
        w,h,c = image.shape
        image = cv2.resize(image,(h*2,w*2))
    
    eyes = advanced_eye_detect(image,eye_cascade,'normal')
    
    if not eyes:
        print('Detection Failed : not human or invalid image')
        continue
        
    rotated_eyes , rotated_images = get_aligned_ROI(image,eyes,)
    
    pivot_x , pivot_y , L = get_aligned_ROI(rotated_images,rotated_eyes , early_stop=True)
    
    ROI = image[pivot_x-(0.6*L):pivot_x+(1.6*L) , pivot_y-(0.6*L):pivot_y+(1.8*L)]
    
    # 여기서부터 지원님꺼랑 붙이면 될듯